In [2]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
import json
import pandas as pd
import datetime
import pytz
import talib

API_KEY = "I0D0ldkssJgACZm3fcs9DlZAX5osmgZp"

from utils import grab_new_data_polygon
from indicators_lib import *

In [3]:
df = grab_new_data_polygon("AAPL")

In [ ]:
 = df['Close']
rsi = talib.ATR(prices, timeperiod=30)

,Volume,VWAP,Open,Close,High,Low,Trades
Date,,,,,,,
28-02-2024 00:00:00 EST,48943139.0,181.1915,182.510,181.42,183.12,180.13,596442
29-02-2024 00:00:00 EST,136682597.0,180.6781,181.270,180.75,182.57,179.53,813073
01-03-2024 00:00:00 EST,73450582.0,179.0322,179.550,179.66,180.53,177.38,911077
04-03-2024 00:00:00 EST,81505451.0,174.8938,176.150,175.10,176.90,173.79,1167166
05-03-2024 00:00:00 EST,94702355.0,170.3234,170.760,170.12,172.04,169.62,1108820
...,...,...,...,...,...,...,...
21-02-2025 00:00:00 EST,51512975.0,246.4026,245.950,245.55,248.69,245.22,508280
24-02-2025 00:00:00 EST,47951624.0,247.2514,244.925,247.10,248.86,244.42,593059
25-02-2025 00:00:00 EST,44242095.0,247.5126,248.000,247.04,250.00,244.91,573942
